In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [8]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band6
  atmpro = mls
  band = 6
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1215
  vmin = 1100
  w_diffuse = [(1.66, 1.66, 1.7, 1.8)]
  wgt = [(0.3, 0.45, 0.6, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312      1.0 -0.000020                  NaN           NaN
0.000312      NaN       NaN                  1.0  1.873108e-06
0.000750      2.0 -0.000019                  2.0  2.552453e-05
0.001052      3.0 -0.000019                  3.0 -1.077952e-05
0.001476      4.0 -0.000019                  4.0 -1.121113e-05
0.002070      5.0 -0.000018                  5.0 -1.146060e-05
0.002904      6.0 -0.000018                  6.0 -1.165693e-05
0.004074      7.0 -0.000017                  7.0 -1.145021e-05
0.005714      8.0 -0.000016                  8.0 -1.164867e-05
0.008015      9.0 -0.000016                  9.0 -1.148164e-05
0.011243     10.0 -0.000015                 10.0 -1.147680e-05
0.015771     11.0 -0.000014                 11.0 -1.141399e-05
0.022122     12.0 -0.000014                 12.0 -1.128775e-05
0.031031     13.0 -0.000013                 13.0 -1.061659e-05
0.043528     14.0 -0.000012                 14.0 -5.787069e-06
0.061057     15.0 -0.000007                 15.0 -3.928240e-06
0.085645     16.0  0.000005                 16.0 -4.681067e-06
0.120136     17.0  0.000028                 17.0 -1.301826e-06
0.168516     18.0  0.000062                 18.0  6.304703e-06
0.236378     19.0  0.000113                 19.0  1.807006e-05
0.331549     20.0  0.000185                 20.0  3.536721e-05
0.465100     21.0  0.000285                 21.0  5.976780e-05
0.652400     22.0  0.000408                 22.0  8.943113e-05
0.915100     23.0  0.000456                 23.0  1.030828e-04
1.283650     24.0  0.000408                 24.0  9.258269e-05
1.800600     25.0  0.000331                 25.0  7.373091e-05
2.525700     26.0  0.000243                 26.0  5.212995e-05
3.542800     27.0  0.000170                 27.0  3.382599e-05
4.969550     28.0  0.000114                 28.0  1.972605e-05
6.970850     29.0  0.000072                 29.0  8.975134e-06
9.778100     30.0  0.000041                 30.0  8.301304e-07
13.715850    31.0  0.000018                 31.0 -5.314646e-06
19.239350    32.0  0.000001                 32.0 -9.940192e-06
26.987250    33.0 -0.000010                 33.0 -1.342907e-05
37.855300    34.0 -0.000018                 34.0 -1.608865e-05
53.100050    35.0 -0.000024                 35.0 -1.848463e-05
73.887500    36.0 -0.000027                 36.0 -1.993890e-05
97.662500    37.0 -0.000027                 37.0 -2.066305e-05
121.437500   38.0 -0.000028                 38.0 -2.263890e-05
145.212500   39.0 -0.000030                 39.0 -2.547414e-05
168.987500   40.0 -0.000033                 40.0 -2.977707e-05
192.762500   41.0 -0.000042                 41.0 -4.394850e-05
216.537500   42.0 -0.000071                 42.0 -9.396393e-05
240.312500   43.0 -0.000071                 43.0 -1.584787e-04
264.087500   44.0  0.000021                 44.0 -2.025284e-04
287.862500   45.0  0.000289                 45.0 -1.508501e-04
311.637500   46.0  0.000815                 46.0  1.166770e-04
335.412500   47.0  0.001649                 47.0  7.365961e-04
359.187500   48.0  0.002777                 48.0  1.807285e-03
382.962500   49.0  0.004037                 49.0  3.195461e-03
406.737500   50.0  0.005660                 50.0  4.920509e-03
430.512500   51.0  0.007695                 51.0  6.880227e-03
454.287500   52.0  0.010193                 52.0  9.016207e-03
478.062500   53.0  0.013196                 53.0  1.133122e-02
501.837500   54.0  0.016716                 54.0  1.396687e-02
525.612500   55.0  0.020812                 55.0  1.720950e-02
549.387500   56.0  0.025490                 56.0  2.146007e-02
573.162500   57.0  0.030794                 57.0  2.695042e-02
596.937500   58.0  0.036796                 58.0  3.373608e-02
620.712500   59.0  0.043533                 59.0  4.163944e-02
644.487500   60.0  0.051121                 6

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -23.814720  0.000000e+00 -23.814720        NaN   
1.000000e-08 1            NaN           NaN        NaN -23.925465   
6.244000e-04 2     -23.814720  1.140301e-09 -23.814720 -23.925465   
8.759000e-04 3     -23.814720  1.608101e-09 -23.814720 -23.925465   
1.228600e-03 4     -23.814720  2.289092e-09 -23.814720 -23.925465   
1.723400e-03 5     -23.814720  3.281216e-09 -23.814720 -23.925465   
2.417400e-03 6     -23.814720  4.726189e-09 -23.814720 -23.925465   
3.390900e-03 7     -23.814720  6.830918e-09 -23.814720 -23.925465   
4.756500e-03 8     -23.814720  9.896017e-09 -23.814720 -23.925465   
6.672000e-03 9     -23.814720  1.435888e-08 -23.814720 -23.925465   
9.358900e-03 10    -23.814720  2.085623e-08 -23.814720 -23.925465   
1.312780e-02 11    -23.814720  3.031421e-08 -23.814720 -23.925465   
1.841450e-02 12    -23.814720  4.408038e-08 -23.814720 -23.925465   
2.583020e-02 13    -23.814720  6.411466e-08 -23.814720 -23.925465   
3.623230e-02 14    -23.814720  9.327152e-08 -23.814720 -23.925465   
5.082340e-02 15    -23.814720  1.361291e-07 -23.814720 -23.925465   
7.129060e-02 16    -23.814720  2.068440e-07 -23.814720 -23.925465   
1.000000e-01 17    -23.814720  3.427695e-07 -23.814720 -23.925465   
1.402710e-01 18    -23.814720  6.155818e-07 -23.814720 -23.925465   
1.967600e-01 19    -23.814720  1.158364e-06 -23.814719 -23.925465   
2.759970e-01 20    -23.814720  2.227689e-06 -23.814718 -23.925465   
3.871000e-01 21    -23.814720  4.313025e-06 -23.814716 -23.925465   
5.431000e-01 22    -23.814719  8.343210e-06 -23.814710 -23.925465   
7.617000e-01 23    -23.814716  1.580980e-05 -23.814700 -23.925465   
1.068500e+00 24    -23.814710  2.725315e-05 -23.814683 -23.925464   
1.498800e+00 25    -23.814704  4.182765e-05 -23.814662 -23.925464   
2.102400e+00 26    -23.814698  5.902252e-05 -23.814639 -23.925464   
2.949000e+00 27    -23.814692  7.793443e-05 -23.814614 -23.925464   
4.136600e+00 28    -23.814689  9.822661e-05 -23.814590 -23.925466   
5.802500e+00 29    -23.814688  1.198450e-04 -23.814568 -23.925469   
8.139200e+00 30    -23.814691  1.427104e-04 -23.814548 -23.925474   
1.141700e+01 31    -23.814699  1.667748e-04 -23.814532 -23.925483   
1.601470e+01 32    -23.814714  1.920053e-04 -23.814522 -23.925495   
2.246400e+01 33    -23.814740  2.183918e-04 -23.814521 -23.925513   
3.151050e+01 34    -23.814778  2.459714e-04 -23.814532 -23.925540   
4.420010e+01 35    -23.814834  2.745078e-04 -23.814559 -23.925577   
6.200000e+01 36    -23.814911  3.006322e-04 -23.814610 -23.925630   
8.577500e+01 37    -23.815006  3.206494e-04 -23.814685 -23.925697   
1.095500e+02 38    -23.815096  3.351569e-04 -23.814761 -23.925765   
1.333250e+02 39    -23.815195  3.543847e-04 -23.814841 -23.925841   
1.571000e+02 40    -23.815315  3.887218e-04 -23.814926 -23.925933   
1.808750e+02 41    -23.815469  4.496725e-04 -23.815019 -23.926052   
2.046500e+02 42    -23.815722  5.835705e-04 -23.815138 -23.926253   
2.284250e+02 43    -23.816349  1.011232e-03 -23.815337 -23.926765   
2.522000e+02 44    -23.817680  2.142456e-03 -23.815537 -23.927886   
2.759750e+02 45    -23.820188  4.708476e-03 -23.815480 -23.930050   
2.997500e+02 46    -23.824491  9.824211e-03 -23.814666 -23.933843   
3.235250e+02 47    -23.831338  1.896562e-02 -23.812372 -23.939985   
3.473000e+02 48    -23.841599  3.387120e-02 -23.807728 -23.949300   
3.710750e+02 49    -23.856067  5.616083e-02 -23.799906 -23.962549   
3.948500e+02 50    -23.874517  8.598151e-02 -23.788536 -23.979551   
4.186250e+02 51    -23.897482  1.248853e-01 -23.772597 -24.000811   
4.424000e+02 52    -23.925463  1.745382e-01 -23.750925 -24.026799   
4.661750e+02 53    -23.958921  2.367040e-01 -23.722217 -24.057981   
4.899500e+02 54    -23.998276  3.132220e-01 -23.685054 -24.094786   
5.137250e+02 55    -24.0

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')